In [12]:
import requests as rq
import psycopg2 as p2
from datetime import datetime, timedelta
import pandas as pd
from pprint import pprint
import os
from dotenv import load_dotenv

##### Знакомство с API

In [28]:
# выясним, как работает API
api_url = 'http://final-project.simulative.ru/data'
params = {'date': 'yyyy-mm-dd'}
response = rq.get(api_url, params=params)
#pprint(response.json())

In [29]:
# видим, что API выдает данные с yyyy-mm-dd вместо даты
# посмотрим на длину полученного json
len(response.json())

7720

In [30]:
# теперь выполним точно такой же запрос несколько раз и посмотрим на длину json
for i in range(4):
    response = rq.get(api_url, params=params)
    print(len(response.json()))
# очевидно, перед нами генератор данных, который подставляет предоставленный параметр date  
# значит, цель - прописать скрипт, который сгенерирует данные за выбранный промежуток времени. Пусть он равняется 6 месяцам.

3090
1988
4066
6178


##### Заполнение базы историческими данными

In [ ]:
# генерируем список интересующих нас дат
start_date = datetime.date(datetime.today() - timedelta(days=183))
end_date = datetime.date(datetime.today() - timedelta(days=1))

date_list = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]

# трансформируем в формат 'yyyy-mm-dd'
date_list_for_script = [f'{i.year}-{i.month:02d}-{i.day:02d}' for i in date_list]

['2026-02-16', '2026-02-17', '2026-02-18', '2026-02-19', '2026-02-20']

In [ ]:
# получаем данные для подключения к БД и настраиваем подключение
result = load_dotenv()
if not result:
    raise EnvironmentError('Не удалось загрузить файл .env или переменные окружения')

db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_pass = os.getenv('DB_PASS')
db_port = os.getenv('DB_PORT')

conn = p2.connect(database=db_name, user=db_user, password=db_pass, host=db_host, port=db_port)

#conn.autocommit = True
cursor = conn.cursor()
conn.status

1

In [ ]:
# формируем базу данных за полгода и загружаем её на сервер
for date in date_list_for_script:
    response = rq.get(api_url, params={'date': date})

Закрыть соединение (не забыть)